In [ ]:
import pandas as pd
from autogluon.timeseries import TimeSeriesPredictor
from autogluon.timeseries.dataset import TimeSeriesDataFrame
from time import time
import torch

torch.cuda.empty_cache()


context_df = pd.read_parquet(
    "https://autogluon.s3.amazonaws.com/datasets/timeseries/electricity_price/train.parquet"
)

test_df = pd.read_parquet(
    "https://autogluon.s3.amazonaws.com/datasets/timeseries/electricity_price/test.parquet"
)

# AutoGluon richiede un TimeSeriesDataFrame per il training (solo inizializzazione)
train_data = TimeSeriesDataFrame.from_data_frame(
    context_df,
    id_column="id",
    timestamp_column="timestamp"
)

predictor = TimeSeriesPredictor(
    prediction_length=24,
    target="target",
    eval_metric="MASE"
)


predictor.fit(
    train_data,
    hyperparameters={
        "Chronos-2": {
            "model_path": "autogluon/chronos-2-small",
            "pretrained": True
        }
    }
)


start = time()
forecast = predictor.predict(TimeSeriesDataFrame.from_data_frame(
    test_df,
    id_column="id",
    timestamp_column="timestamp"
))
elapsed = time() - start

print(forecast)
print(f"Tempo per la predizione: {elapsed:.2f} secondi")